In [1]:
# Importa
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import os
from tqdm import tqdm
import sys

In [2]:
# Carica il file CSV
file_path = './consultori-roma-mail-orari.csv'
data = pd.read_csv(file_path)

In [3]:
# Crea l'oggetto geocoder
geolocator = Nominatim(user_agent="consultori_mapper")

In [4]:
# Creazione della mappa di Roma
roma_map = folium.Map(location=[41.9028, 12.4964], zoom_start=12)  # Coordinate centrali di Roma

In [5]:
# Definisci colori per le diverse ASL
asl_colors = {
    'ROMA 1': 'blue',
    'ROMA 2': 'green',
    'ROMA 3': 'red',
    'ROMA 4': 'purple',
    'ROMA 5': 'orange',
    'ROMA 6': 'darkred'
}

In [6]:
# Aggiunta di marker per ogni consultorio
for index, row in tqdm(data.iterrows(), total=len(data), desc="Aggiunta marker consultori", file=sys.stdout, ncols=100):
    nome_consultorio = row['Consultorio Familiare']
    asl = row['ASL']
    indirizzo = row['Indirizzo']
    telefono = row['Telefono']
    orari = row['Orari']
    contact = row['Email/sito']

    # Ottieni le coordinate usando il geocoder
    try:
        location = geolocator.geocode(indirizzo + ", Roma")
        if location:
            lat, lon = location.latitude, location.longitude

            # Creazione del contenuto del popup
            if '@' in contact or contact == "n/d":
                popup_content = f"""
                          <h4>{nome_consultorio}</h4>
                          <i>{asl}</i>
                          <br>
                          <ul>
                              <li style='list-style-type: square'><b>Indirizzo</b>: {indirizzo}
                              <li style='list-style-type: square'><b>Telefono</b>: {telefono}
                              <li style='list-style-type: square'><b>Email/sito</b>: {contact}
                          </ul>
                          <p>Orari apertura: <br>
                          {orari}
                          </p>
                            """
            else:
                popup_content = f"""
                          <h4>{nome_consultorio}</h4>
                          <i>{asl}</i>
                          <br>
                          <ul>
                              <li style='list-style-type: square'><b>Indirizzo</b>: {indirizzo}
                              <li style='list-style-type: square'><b>Telefono</b>: {telefono}
                              <li style='list-style-type: square'><b>Email/sito</b>: <a href="https://{contact}" target="_blank">{contact}</a>
                          </ul>
                          <p>Orari apertura: <br>
                          {orari}
                          </p>
                            """

            # Aggiungi un marker per il consultorio
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_content, max_width=500),
                icon=folium.Icon(color=asl_colors.get(asl, 'gray'), icon='info-sign')
            ).add_to(roma_map)

            # Clear the line to avoid clutter
            sys.stdout.write("\033[F")
    except Exception as e:
        print(f"Errore nel geocoding per {indirizzo}: {e}")

Aggiunta marker consultori: 100%|███████████████████████████████████| 83/83 [00:45<00:00,  1.83it/s]


In [7]:
# Aggiungi una legenda alla mappa
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: fit-content; height: fit-content; 
            background-color: white; z-index:9999; font-size:14px; opacity: 0.8; border-radius: 6px;
            border:2px solid grey; padding: 10px;">
    <b>Legenda ASL</b><br>
    <i class="fa fa-map-marker fa-2x" style="color:blue"></i> ROMA 1<br>
    <i class="fa fa-map-marker fa-2x" style="color:green"></i> ROMA 2<br>
    <i class="fa fa-map-marker fa-2x" style="color:red"></i> ROMA 3<br>
    <i class="fa fa-map-marker fa-2x" style="color:purple"></i> ROMA 4<br>
    <i class="fa fa-map-marker fa-2x" style="color:orange"></i> ROMA 5<br>
    <i class="fa fa-map-marker fa-2x" style="color:darkred"></i> ROMA 6<br>
</div>
'''
roma_map.get_root().html.add_child(folium.Element(legend_html))

In [8]:
# Salvataggio della mappa in un file HTML nella stessa directory del file CSV
output_path = os.path.join(os.path.dirname(file_path), 'consultori_roma_map.html')
roma_map.save(output_path)
print(f"Mappa salvata come {output_path}")

Mappa salvata come ./consultori_roma_map.html
